In [49]:
%matplotlib inline
import matplotlib.pyplot as plt
#import graphviz
import numpy as np
import pandas as pd
import xgboost as xgb
import sys
import kaggle_xgb
import calc_ndcg
import itertools
#import ftr_eng
#reload(ftr_eng)
#from sklearn.grid_search import ParameterGrid

Read in just the testing and training data.

In [4]:
#train_subset = pd.read_csv('../data/train_users_2.csv', nrows = 100000, index_col='id')
train_data = pd.read_csv('../data/train_users_2.csv', index_col = 'id')
test_data = pd.read_csv('../data/test_users.csv', index_col = 'id')
#sessions = pd.read_csv('../data/sessions.csv')
#age = pd.read_csv('../data/age_gender_bkts.csv')
#countries = pd.read_csv('../data/countries.csv')

In [63]:
test_data.query('1000>age>110')

,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
id,,,,,,,,,,,,,,


In [83]:
len(train_data.query('age<16'))

65

In [80]:
train_data.query('age==16').country_destination.value_counts(normalize = True)

NDF    0.961538
US     0.038462
Name: country_destination, dtype: float64

In [54]:
train_data[pd.isnull(train_data.age)].country_destination.value_counts(normalize = True)

NDF      0.768428
US       0.156529
other    0.028617
FR       0.014888
IT       0.009081
GB       0.006251
ES       0.006171
CA       0.003989
DE       0.002387
NL       0.001818
AU       0.001171
PT       0.000671
Name: country_destination, dtype: float64

In [55]:
train_data.query('age17').country_destination.value_counts()

NDF      0.738462
US       0.138462
IT       0.030769
FR       0.030769
DE       0.015385
ES       0.015385
GB       0.015385
other    0.015385
Name: country_destination, dtype: float64

In [3]:
test_sessions = sessions[sessions.user_id.isin(test_data.index)]
tr_sessions = sessions[sessions.user_id.isin(train_data.index)]
tr_sessions.set_index('user_id', drop= False, inplace=True)
tr_sessions = pd.concat([tr_sessions, train_data.country_destination], join = 'inner', axis = 1)

In [92]:
train_df = train_data.loc[train_data.index.isin(tr_sessions.user_id)]

In [93]:
train_df = ftr_eng.sessions(train_df, tr_sessions)
train_df = ftr_eng.users(train_df)
train_df = ftr_eng.one_hot(train_df)
train_Y = ftr_eng.binarize_targets(train_df)

number of columns before one hot encoding 30
number of columns after one hot encoding 139


In [104]:
train_X = train_df2.drop('country_destination', axis = 1).values

In [105]:
#baseline cross-valdation
param = {'objective': 'binary:logistic', 'seed': 0}
param['eta'] = 0.20
param['max_depth'] = 5
param['subsample'] = .5
param['col_sample_bytree'] = .6
results = {}

dtrain = xgb.DMatrix(train_X, label = targets, missing = -1)
baseline_sessions = xgb.cv(param, dtrain, num_boost_round = 40, nfold = 10, metrics={'error'})

In [107]:
bst = xgb.train(param, dtrain, num_boost_round = 40)

In [110]:
features = sorted(bst.get_fscore().items(), key = lambda x: x[1], reverse = True)
[(feat, train_df2.drop('country_destination', axis = 1).columns[int(feat[1:])], score) for feat, score in features]

[('f4', 'secs_elapsed', 190),
 ('f0', 'age', 114),
 ('f3', 'action_counts', 114),
 ('f12', 'dac_day', 83),
 ('f16', 'tfa_hour', 62),
 ('f2', 'message_post', 47),
 ('f1', 'booking_request', 33),
 ('f13', 'dac_dayofweek', 25),
 ('f17', 'gender_-unknown-', 21),
 ('f56', 'affiliate_channel_content', 18),
 ('f101', 'first_browser_-unknown-', 17),
 ('f7', 'Windows Desktop', 16),
 ('f22', 'signup_method_facebook', 16),
 ('f5', 'update_listing', 14),
 ('f18', 'gender_FEMALE', 14),
 ('f21', 'signup_method_basic', 13),
 ('f80', 'first_affiliate_tracked_-1', 11),
 ('f57', 'affiliate_channel_direct', 11),
 ('f36', 'language_en', 11),
 ('f95', 'first_device_type_Mac Desktop', 10),
 ('f113', 'first_browser_IE', 10),
 ('f9', 'Android Phone', 9),
 ('f107', 'first_browser_Chrome', 9),
 ('f84', 'first_affiliate_tracked_omg', 9),
 ('f8', 'iPhone', 8),
 ('f87', 'first_affiliate_tracked_untracked', 8),
 ('f81', 'first_affiliate_tracked_linked', 8),
 ('f24', 'signup_flow_0', 8),
 ('f29', 'signup_flow_24', 8

In [106]:
baseline_sessions

,test-error-mean,test-error-std,train-error-mean,train-error-std
0,0.242677,0.003862,0.240481,0.000529
1,0.240862,0.004603,0.239508,0.000904
2,0.241526,0.003424,0.239112,0.001017
3,0.240293,0.003416,0.238420,0.001099
4,0.240509,0.003280,0.238527,0.000744
5,0.240225,0.003642,0.238348,0.000681
6,0.239575,0.003936,0.237926,0.000651
7,0.239981,0.004520,0.237604,0.000618
8,0.239629,0.003832,0.237101,0.000505
9,0.239249,0.004102,0.237064,0.000702


In [103]:
train_df2 = train_df.drop(['dac_month', 'tfa_month'], axis = 1)

In [ ]:
post_cnts = tr_sessions.groupby(['user_id']).apply(lambda x: np.sum(x['action_type'] == 'message_post'))

In [ ]:
train_df = pd.concat([train_df, posters_cnts], axis = 1, join = 'inner')
train_df = train_df.rename(columns = {0:'post_cnts'})

In [ ]:
axes = plt.gca()
axes.set_xlim([0,20000])
#s.hist(ax=axes, bins= 1000)
tr_sessions[tr_sessions.action == 'this_hosting_reviews']['secs_elapsed'].hist(bins = 1000)

In [6]:
acts = tr_sessions.action_detail.unique()
acts = [x for x in acts if x is not np.NaN]
for x in acts:
    if 'thread' in x:
        print x

message_thread
toggle_archived_thread
lookup_message_thread
toggle_starred_thread


In [ ]:
#31358 users posted messages in test set, it's worth distinguishing NDF and non-NDF?
#fifty-fifty split
print len(test_sessions.query('action_type == "message_post"').index.unique())
posters = tr_sessions.query('action_type == "message_post"').index.unique()
train_data.loc[posters, 'country_destination'].value_counts(normalize = True)

In [ ]:
print len(test_sessions.query('action_type == "booking_request"').index.unique())
train_data.loc[all_bookers, 'country_destination'].value_counts(normalize = True)

In [ ]:
#8067 booking requests in test set, it's worth distinguishing NDF and non-NDF
print len(test_sessions.query('action_type == "booking_request"').index.unique())
train_data.loc[all_bookers, 'country_destination'].value_counts(normalize = True)

In [ ]:
all_bookers = tr_sessions.query('action_type == "booking_request"').index.unique()
users_NDF = tr_sessions.query('action_type == "booking_request" & country_destination == "NDF"').index.unique()
users_USA = tr_sessions.query('action_type == "booking_request" & country_destination == "US"').index.unique()
#tr_sessions[tr_sessions.action_type == 'booking_request'].groupby('user_id').filter(lambda x: len('x.country_destination')>1)

In [8]:
cancel_NDF = tr_sessions.query('action == "cancel" & country_destination != "NDF"').user_id

In [35]:
#set hierarchical index???
x = tr_sessions.groupby('user_id').count().country_destination
#groupby reorders user_id, so put back in the original order
x = x.loc[tr_sessions.user_id.unique()]
pos = [range(i) for i in x.tolist()]
pos = list(itertools.chain.from_iterable(pos))
tr_sessions['pos'] = pos
len(pos) == len(tr_sessions)

True

In [36]:
#position normalized by total number of actions
pos_norm = [np.arange(total)/float(total-1) for total in x.tolist()]
pos_norm = list(itertools.chain.from_iterable(pos_norm))
tr_sessions['pos_norm'] = pos_norm
len(pos_norm) == len(tr_sessions)

True

In [ ]:
len(test_sessions[test_sessions.action_type == 'booking_request'].groupby('user_id'))

In [ ]:
test_sessions[test_sessions.action == 'add_guests'].groupby('user_id').count()

In [ ]:
tr_sessions[tr_sessions.action == 'add_guests'].groupby('user_id')['country_destination'].unique()

In [ ]:
actions_both30 = set(tr_sessions.action.unique()) & set(actions_30.action.unique())

In [ ]:
actions_30 = test_sessions.groupby('action').filter(lambda x: x['user_id'].nunique()>30)

In [79]:
getcntr('ajax_payout_split_edit', tr_sessions.action, train_data)

NDF    5
Name: country_destination, dtype: int64

In [ ]:
getcntr('ajax_google_translate', tr_sessions, train_data)

In [53]:
tr_sessions[tr_sessions.user_id == 'gg6b14vkqh'].action.unique()

array(['similar_listings', 'ajax_lwlb_contact', 'ajax_refresh_subtotal',
       'ajax_check_dates', nan, 'ask_question', 'index', 'search_results',
       'lookup', 'show', 'personalize', 'other_hosting_reviews_first',
       'identity', 'jumio_token', 'jumio_redirect', 'create', 'verify',
       'request_new_confirm_email', 'confirm_email', 'complete_status',
       'travel_plans_current', 'dashboard', 'header_userpic'], dtype=object)

In [42]:
getcntr('travel_plans_current', tr_sessions.action, train_data)

US       4349
NDF      2477
other     737
FR        303
IT        232
GB        154
ES        135
CA        114
DE         57
NL         56
AU         28
PT         18
Name: country_destination, dtype: int64

In [ ]:
tr_sessions[tr_sessions.user_id == 'd1mm9tcy42'].action_detail.unique()

In [ ]:
tr_sessions[tr_sessions.user_id == 'jluu5qcwhg'].action_detail.unique()

tr_sessions[tr_sessions.user_id == 'qtw88d9pbl'].action_detail.unique()

In [34]:
tr_sessions[tr_sessions.action_detail == 'change_trip_characteristics']

,user_id,action,action_type,action_detail,device_type,secs_elapsed,country_destination
d1mm9tcy42,d1mm9tcy42,ajax_refresh_subtotal,click,change_trip_characteristics,Windows Desktop,8134,other
d1mm9tcy42,d1mm9tcy42,ajax_refresh_subtotal,click,change_trip_characteristics,Windows Desktop,175570,other
d1mm9tcy42,d1mm9tcy42,ajax_refresh_subtotal,click,change_trip_characteristics,Windows Desktop,982,other
d1mm9tcy42,d1mm9tcy42,ajax_refresh_subtotal,click,change_trip_characteristics,Windows Desktop,1013,other
d1mm9tcy42,d1mm9tcy42,ajax_refresh_subtotal,click,change_trip_characteristics,Windows Desktop,427,other
d1mm9tcy42,d1mm9tcy42,ajax_refresh_subtotal,click,change_trip_characteristics,Windows Desktop,212772,other
d1mm9tcy42,d1mm9tcy42,ajax_refresh_subtotal,click,change_trip_characteristics,Windows Desktop,42050,other
d1mm9tcy42,d1mm9tcy42,ajax_refresh_subtotal,click,change_trip_characteristics,Windows Desktop,791,other
ncf87guaf0,ncf87guaf0,ajax_refresh_subtotal,click,change_trip_characteristics,Windows Desktop,3664,NDF
ncf87guaf0,ncf87guaf0,ajax_refresh_subtotal,click,change_trip_characteristics,Windows Desktop,3913,NDF


In [ ]:
getcntr('update_listing_description', tr_sessions.action_detail, train_data)

In [ ]:
getcntr('ajax_refresh_subtotal', tr_sessions.action, train_data)

In [27]:
def testcnts(q, test_series):
    users = test_series[test_series == q].index.unique()
    return len(users)

def getcntr(q, tr_series, train_data):
    users = tr_sessions[tr_series == q].index.unique()
    return train_data.loc[users].country_destination.value_counts()

def getcntr2(q1, q2, tr_series, train_data):
    users1 = tr_sessions[tr_series == q1].index.unique()
    users2 = tr_sessions[tr_series == q2].index.unique()
    users = set(user1) & set(users2)
    return train_data.loc[users].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'travel_plans_current'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'place_worth'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'payout_update'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'guest_billing_receipt'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'booking'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'guest_booked_elsewhere']

In [ ]:
tr_sessions[tr_sessions.action == 'guest_booked_elsewhere'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'transaction_history'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'receipt'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'countries_option'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'itinerary'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'travel_plans_previous'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'cancel'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'office_location'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action_detail == 'your_trips'].country_destination.value_counts()

In [ ]:
users

In [ ]:
users = tr_sessions.groupby('action')['user_id'].unique()['Mac Desktop']

In [ ]:
tr_sessions[tr_sessions.user_id.isin(users)]

In [ ]:
users = tr_sessions[tr_sessions.action == 'host_summary'].user_id

In [ ]:
tr_sessions[tr_sessions.action == 'phone_number_widget'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'payment_instruments'].country_destination.value_counts()

In [ ]:
tr_sessions[tr_sessions.action == 'agree_terms_uncheck'].country_destination.value_counts(normalize = True)

In [ ]:
test_sessions[test_sessions.action == 'print_confirmation']

In [ ]:
options = ['recommend', 'recommendations', 'references', 'referrer_status']
for i in options:
    print tr_sessions[tr_sessions.action == i].country_destination.value_counts()

In [ ]:
'pay',
 'payment_instruments',
 'payment_methods',
 'payout_update',
 'payout_update'

In [ ]:
actions_both30

In [ ]:
axes = plt.gca()
axes.set_xlim([0,20000])
s.hist(ax=axes, bins= 10000)

In [ ]:
s2 = tr_sessions.groupby('user_id').filter(lambda x: len(x['action'])>3)

In [ ]:
a = [1, 2, 3, 4]
a[-1:-3]

In [ ]:
s2.groupby('user_id').apply(lambda x: x.iloc[1:2, :] )

In [ ]:
s2[s2.country_destination == 'NDF'].action.value_counts()

In [ ]:
s = sessions.groupby('user_id')['secs_elapsed'].apply(lambda x:  x.iloc[-1])

In [ ]:
sessions.dropna(subset=['user_id'], inplace = True)

sessions_tr = sessions[sessions.user_id.isin(train_data.index)]
sessions_test = sessions[sessions.user_id.isin(test_data.index)]
a_ignore = set(sessions_tr.action.unique()) ^ set(sessions_test.action.unique())
#sessions.groupby('action')['action_detail']

#set(sessions_test.action_detail.unique()) ^ set(sessions_tr.action_detail.unique())
#sessions_test.action_detail.unique()
#sessions_tr.action_detail.unique()
#train_data['date_account_created'] = pd.to_datetime(train_data.date_account_created)
#x = train_data[train_data.date_account_created>='2014']
#train_data[train_data.index.isin(sessions_tr.user_id)].timestamp_first_active

In [ ]:
actions50 = [sessions_test.groupby('action')['user_id'].nunique() >50]
print 'Number of unique actions with more than 50 users:', np.sum(sessions_test.groupby('action')['user_id'].nunique()>50)

In [ ]:
#Let's get the actions with just one action type and one action detail.
print 'Number of unique actions with just one action type:',
np.sum(sessions.groupby('action')['action_detail'].nunique()==1)
#Take the intersection of these actions with a_ignore.

In [ ]:
sessions['action_type'] = sessions['action_type'].fillna('missing')
sessions['action_detail'] = sessions['action_detail'].fillna('missing')

In [ ]:
#actions with null values
actions_null2 = sessions[pd.isnull(sessions.action_type)].action.unique()
actions_null == actions_null2

In [ ]:
df = pd.DataFrame({'A': ['foo','bar','foo','foo', 'zoo','bar','foo','foo'],
                       'B': ['coo', np.NaN,'coo','coo', 'dude', np.NaN,'fooo','fooo']})

In [ ]:
def custom_filter(x):
     return (x['action_detail'].nunique() > 1) or (x['action_type'].nunique() > 1) 
actions_df = sessions.groupby('action').filter(lambda x: custom_filter(x))

In [ ]:
tr_sessions[tr_sessions.action == 'update'].action_detail.unique()

In [ ]:
actions_df.action.unique()

In [ ]:
def custom_filter(x):
     return (x['action_detail'].nunique() == 1) and (x['action_type'].nunique() == 1) 
#get all actions with just one action_type and action 
actions_df = sessions.groupby('action').filter(lambda x: custom_filter(x))

In [ ]:
#exclude those actions that are found in one table or the other
actions_df = actions_df[~actions_df.action.isin(a_ignore)]

In [ ]:
actions_df.action[pd.isnull(actions_df['secs_elapsed'])].value_counts()

In [ ]:
#info = actions_1.groupby(['user_id', 'action']).apply(lambda x: x['secs_elapsed'].sum())
action_cnts = actions_df.groupby(['user_id', 'action']).apply(lambda x: x['device_type'].count())

In [ ]:
action_cnts = action_cnts.unstack(level=-1)

In [ ]:
action_cnts.fillna(0, inplace=True)

In [ ]:
pd.to_pickle(action_cnts, '../data/actions.p')

In [ ]:
#features with no time
#null_time, cnts = np.unique(actions_1[pd.isnull(actions_1['secs_elapsed'])].action, return_counts= True)
#zip(cnts, null_time)

In [ ]:
#find the number of users in test data with an action that is not found in the training sessions data
#We should look to find similar items to substitute them
for i in a_ignore:
    users = sessions_test[sessions_test.action == i].user_id.unique()
    if len(users) > 0:
        print i, len(users)

In [ ]:
actions_df.action_type.unique()

In [ ]:
#iterate through each action and find those that have more than one action detail/ action type
#s = sessions.groupby('action')['action_type'].nunique()
#sessions.groupby('action')['action_detail'].nunique()

In [ ]:
def ndcg(preds, labels):
    """Calculate sum of normalzied discounted cumulative gain for the predictions
    The correct prediction will have a relevance of 1, incorrect predictions will have a relevance of 0.
    Weight the relevance values such that it is reduced logarithmically proportional to it its position     
    
    Args:
        preds: n*5 array of predictor targets
        labels: n*1 array targets
    Returns:
        sum of normalized discounted cumulative gain for all the predictions
    """
    
    #find positions where the prediction matches the label
    relv_pos = np.where(np.equal(preds, labels))[1]
    #weight func: log2(i+1), add one more to adjust for 0-based indexing
    total_ndcg = np.sum(1./np.log2(relv_pos+2))
    return total_ndcg
    
def evalerror(cls_prob, dtrain):
    """find top k predictions from probability matrix and call ndcg to find accuracy of predictions
    
    Args:
        cls_prob: 2D array, probability of each class for each person (n persons by m classes),
                  the column index corresponds with class
        labels: labels for the n persons
    returns:
        prediction accuracy using ndcg to evaluate predictions of each AirBNB user
    """
    #determine the top k predictions
    labels = dtrain.get_label()
    k = 5
    top_k = cls_prob.argsort(axis = 1)[:,:k:-1]
    #convert true values  and compared with predictions to check for equality
    labels = labels[:, None]
    return 'error', 1.-ndcg(top_k, labels)/len(labels)

In [ ]:
features = sorted(bst.get_fscore().items(), key = lambda x: x[1], reverse = True)
[(feat, all_df.columns[int(feat[1:])], score) for feat, score in features]